In [2]:
!pip install opencv-python

In [5]:
import os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import cv2
from sklearn.model_selection import train_test_split

dataset_path = "../dataset/lfw_funneled/"

In [2]:
# from imutils import paths
# imagePaths = list(paths.list_images(dataset_path))
# imagePaths

In [6]:
data = []
for person in os.listdir(dataset_path):
    path = os.path.join(dataset_path,person)
    label = person
    
    for img in os.listdir(path):
        imgPath = os.path.join(path, img)
        person_img = cv2.imread(imgPath,0)
        try:
            person_img = cv2.resize(person_img,(50,50))
            image = np.array(person_img).flatten()
            data.append([image, label])
        except Exception as e:
            pass
        
print(len(data))

13233


In [7]:
data

[[array([ 0,  0,  0, ..., 36, 34, 19], dtype=uint8), 'Walter_Mondale'],
 [array([10, 10, 11, ..., 14, 13, 13], dtype=uint8), 'Walter_Mondale'],
 [array([0, 0, 0, ..., 6, 0, 0], dtype=uint8), 'Walter_Mondale'],
 [array([0, 0, 0, ..., 0, 0, 0], dtype=uint8), 'Walter_Mondale'],
 [array([0, 0, 0, ..., 0, 0, 0], dtype=uint8), 'Walter_Mondale'],
 [array([0, 0, 0, ..., 0, 1, 0], dtype=uint8), 'Walter_Mondale'],
 [array([0, 0, 0, ..., 0, 0, 0], dtype=uint8), 'Walter_Mondale'],
 [array([167, 167, 167, ...,  38,  36,  31], dtype=uint8), 'Walter_Mondale'],
 [array([101,  99,  97, ...,  20,  20,  11], dtype=uint8), 'Walter_Mondale'],
 [array([ 0,  0,  0, ..., 35, 38, 39], dtype=uint8), 'Walter_Mondale'],
 [array([24, 24, 24, ..., 43, 36, 17], dtype=uint8), 'Katrin_Susi'],
 [array([  4, 230, 226, ...,  39,  49,  30], dtype=uint8),
  'Melissa_Etheridge'],
 [array([ 0,  3, 20, ...,  3,  3,  3], dtype=uint8), 'Melissa_Etheridge'],
 [array([ 2,  2,  1, ..., 50, 50,  0], dtype=uint8), 'Yuri_Luzhkov'],
 

### Read for number of people and number of images of each individual